# 0.安装Isaac Sim

> Isaac Sim的相关包路径无法正确地被VS Code之类的IDE识别，这是因为Isaac Sim的相关包并非完全按照标准Python包组织，因此需要添加脚本来设置环境，使得在正常的编辑器环境中，Isaac Sim的相关包能够被找到。打开终端，进入到你的项目的根目录下执行这个脚本，可以让编辑器的python分析正确地识别Isaac Sim相关包。[参考:Nvidia Isaac Sim安装与配置 入门教程 2024（2）](https://blog.csdn.net/m0_56661101/article/details/139772773)、[脚本仓库](https://github.com/Auromix/auro_sim)
>
> `script_name="setup_isaac_sim_ide_environment.sh" && directory_name="/tmp" && download_url="https://raw.githubusercontent.com/Auromix/auro_sim/main/scripts" && full_script_path="${directory_name}/${script_name}" && wget -O $full_script_path $download_url/$script_name && sudo chmod +x $full_script_path && bash $full_script_path && rm -rf $full_script_path
`

- [Nvidia Isaac Sim图形界面 入门教程 2024（3）](https://blog.csdn.net/m0_56661101/article/details/139826487)
- [Nvidia Isaac Sim物理场景、材质和USD 入门教程 2024（4）](https://blog.csdn.net/m0_56661101/article/details/139831209)
  > 为了更好地管理加载的USD文件和避免加载原始世界的环境内容，USD中存在名为defaultPrim的概念，在一个USD文件被其他USD文件引用时，只有defaultPrim部分会被拉取。
- [Nvidia Isaac Sim组装机器人和添加传感器 入门教程 2024（5）](https://blog.csdn.net/m0_56661101/article/details/139840541)
- [Nvidia Isaac Sim图编程OmniGraph 入门教程 2024（6）](https://blog.csdn.net/m0_56661101/article/details/139919794) 可以跳过，直接看后面的教程
- [Nvidia Isaac Sim代码编程 入门教程 2024（7）](https://blog.csdn.net/m0_56661101/article/details/139936671)、[脚本仓库](https://github.com/Auromix/auro_sim) 看完# 4.Interactive Scripting再看

# 1.Omniverse Isaac Sim Glossary术语表


[Glossary术语表](https://docs.omniverse.nvidia.com/isaacsim/latest/reference_glossary.html#isaac-sim-glossary-prim)

[Isaac Sim Workflows](https://docs.omniverse.nvidia.com/isaacsim/latest/introductory_tutorials/tutorial_intro_workflows.html)
- GUI
- Extensions
- Standalone Python

In [ ]:
#launch Isaac Sim before any other imports
#default first two lines in any standalone application
from isaacsim import SimulationApp
simulation_app = SimulationApp({"headless": False}) # we can also run as headless.

from omni.isaac.core import World
world = World()
# world.scene.add_default_ground_plane()
# add

# Its recommended to always do a reset after adding your assets, for physics handles to be propagated properly
# Resetting the world needs to be called before querying anything related to an articulation specifically.
world.reset()

for i in range(100):

    # func

    world.step(render=True) # execute one physics step and one rendering step

simulation_app.close() # close Isaac Sim

[Jupyter Notebook](https://docs.omniverse.nvidia.com/isaacsim/latest/core_api_tutorials/tutorial_core_hello_world.html#converting-the-example-to-a-standalone-application-using-jupyter-notebook)

# 2.Omnigraph

#### 2.1
[Action Graph Quickstart](https://docs.omniverse.nvidia.com/extensions/latest/ext_omnigraph/tutorials/quickstart.html)
- 一个类似blender的例子。
  - First, load the `Action Graph Bundle extension` into Omniverse USD Composer
  - Use a `Write Prim Attribute Node`: <font color=GreenYellow>**Drag**</font> the prim from the Stage to the Action Graph editor
  - `flipflop node`

[OmniGraph](https://docs.omniverse.nvidia.com/isaacsim/latest/gui_tutorials/tutorial_gui_omnigraph.html#)

- Action Graph vs Push Graph: **For majority of cases in Isaac Sim, you will be using the Action Graph.**
  > An Action graph, or sometimes called an “Execution Graph”, is executed whenever an execution node gets triggered. In many Isaac Sim cases, the trigger is a “tick” node that is set to be triggered every simulation frame tick, so that the graph doesn’t do anything when simulation isn’t running, and ticks at every rendering frame once simulation starts. If you remove the tick node from an action graph, the graph will not run upon pressing “play”.
  >
  > Action Graphs can have other triggers, such as a keyboard/mouse input, or a stage event. You can find the relevant nodes under the “Event” nodes category.
  >
  > A Push graph, or sometimes “Generic Graph”, will execute automatically on every rendering frame, without needing an “execution” node. Since the default viewport is always running and therefore rendering, any nodes you drag into a Push Graph editor will immediately start executing, even if the simulation isn’t “playing”. You can test this by dragging a “Print Text” node into a Push Graph editor, open your console tab and allow it to publish Informational level logs. Then the moment you finish typing something in the “Text” section of the print node, the text will get printed.
- jetbot法1，手动链接graph node
- `On Playback Tick` node emits an execution event for every frame, but only while the simulation is playing.
  - `On Tick` node executes an output execution pulse at a regular multiple of the refresh rate.[ OmniGraph Node Library: On Tick](https://docs.omniverse.nvidia.com/extensions/latest/ext_omnigraph/node-library/nodes/omni-graph-action/ontick-1.html)
- 两个外设输入例子：
Isaac Examples > Input Devices > Kaya Gamepad/Omnigraph Keyboard，在stage中xx_graph上右键单击**Open Graph**可以查看编辑

#### 2.2
进一步的封装常用功能

[Commonly Used Omnigraph Shortcuts](https://docs.omniverse.nvidia.com/isaacsim/latest/advanced_tutorials/tutorial_advanced_omnigraph_shortcuts.html#commonly-used-omnigraph-shortcuts)

jetbot法2【articulation controller】【differential Controller】

#### 2.3
Prerequisites，先看看# 4.Interactive Scripting 交互式脚本

[scripting OmniGraph via Python](https://docs.omniverse.nvidia.com/isaacsim/latest/advanced_tutorials/tutorial_advanced_omnigraph_scripting.html#isaac-sim-app-tutorial-advanced-omnigraph-scripting)

In [ ]:
import omni.graph.core as og

keys = og.Controller.Keys
(graph_handle, list_of_nodes, _, _) = og.Controller.edit(
    {"graph_path": "/action_graph", "evaluator_name": "execution"},
    {
        keys.CREATE_NODES: [
            ("tick", "omni.graph.action.OnTick"),
            ("print","omni.graph.ui_nodes.PrintText")
        ],
        keys.SET_VALUES: [
            ("print.inputs:text", "Hello World"),
            ("print.inputs:logLevel","Warning")                 # setting the log level to warning so we can see the printout in terminal
        ],
        keys.CONNECT: [
            ("tick.outputs:tick", "print.inputs:execIn")
        ],
    },
)

In [ ]:
# get existing value from an attribute
existing_text = og.Controller.attribute("/action_graph/print.inputs:text").get()
print("Existing Text: ", existing_text)

# set new value
og.Controller.attribute("/action_graph/print.inputs:text").set("New Texts to print")

In [ ]:
og.Controller.create_node("/action_graph/new_node_name", "omni.graph.nodes.ConstantString")
og.Controller.attribute("/action_graph/new_node_name.inputs:value").set("This is a new node")
og.Controller.connect("/action_graph/new_node_name.inputs:value", "/action_graph/print.inputs:text")

有一个特殊参数称为“pipeline_stage”，可以将Graph设置为“按需”执行，仅在调用时执行来显式触发而不是在仿真的每一帧上进行。

通常，在创建图形时设置此变量，如下所示`"pipeline_stage": og.GraphPipelineStage.GRAPH_PIPELINE_STAGE_ONDEMAND`。

In [ ]:
(demand_graph_handle,_,_,_) = og.Controller.edit(
    {
        "graph_path": "/ondemand_graph",
        "evaluator_name": "execution",
        "pipeline_stage": og.GraphPipelineStage.GRAPH_PIPELINE_STAGE_ONDEMAND
    },
    {
        keys.CREATE_NODES: [
            ("tick", "omni.graph.action.OnTick"),
            ("print","omni.graph.ui_nodes.PrintText")
        ],
        keys.SET_VALUES: [
            ("print.inputs:text", "On Demand Graph"),
            ("print.inputs:logLevel","Warning")
        ],
        keys.CONNECT: [
            ("tick.outputs:tick", "print.inputs:execIn")
        ],
    },
)

或者，您也可以通过以下方式为已经创建的Graph设置它：`yourGraph_handle.change_pipeline_stage(og.GraphPipelineStage.GRAPH_PIPELINE_STAGE_ONDEMAND)`

测试：
Start simulation by press “play”. 
To manually trigger a graph, open another tab in the Script Editor, and paste in`yourGraph_handle.evaluate()`

> 有关将图形附加到物理回调和/或渲染回调的更深入示例，请参见`standalone_examples/api/omni.isaac.core/omnigraph_triggers.py`

#### 2.4

Omnigraph’s Custom Node Tutorial 【example文件没找到，可以看看图文】

- [Part One](https://docs.omniverse.nvidia.com/extensions/latest/ext_omnigraph/tutorials/deforming_text_run_graph.html)、[Part Two](https://docs.omniverse.nvidia.com/extensions/latest/ext_omnigraph/tutorials/deforming_text_create_graph.html)、[Part Three：Create Your Own Nodes](https://docs.omniverse.nvidia.com/extensions/latest/ext_omnigraph/tutorials/deforming_text_create_node.html)

[创建自己的 Omnigraph（以ros2消息发布节点为例）](https://blog.csdn.net/Vulcan_S/article/details/140767393)
> 官方为了我们更好理解摄像头和雷达的OmniGraph，开源了几个例子，在 isaac_sim 目录 exts\omni.isaac.ros2_bridge 里面以及 exts\omni.isaac.sensor 里面有激光等传感器，如 OgnIsaacPrintRTXLidarInfo 可以处理并打印原始RTX激光雷达数据，OgnROS2RtxLidarHelper 可以看到如何使用 replicator 发布激光点云数据，也可以直接在这些例程上修改。

# 3.Attach a Camera to Robot

[Attach a Camera to Robot](https://docs.omniverse.nvidia.com/isaacsim/latest/gui_tutorials/tutorial_gui_camera_sensors.html#attach-a-camera-to-robot)

# 4.Interactive Scripting 交互式脚本

1. Script Editor 内部的脚本编辑器 
   
   通过在脚本编辑器下的标签菜单中打开多个标签。所有标签共享相同的环境，因此在一个环境中导入的库或定义的变量可以在其他环境中访问和使用。
2. REPL Extension
   
   启用Isaac Sim REPL扩展，打开一个新的终端，在命令行中运行 `telnet localhost 8223`。终端中会启动一个Python shell，
   这样就可以在shell里通过 Python 与在 Isaac Sim 中打开的Stage进行交互。
3. Raw USD APIs
   > 注意：Raw USD APIs are versatile and detailed but complex, especially for beginners. 
4. Isaac Sim Core APIs !
   > Isaac Sim has a set of core APIs that simplify some of the frequently used actions for robotics simulators. These APIs abstract away default parameter settings.
   > 
   > Using Isaac Sim Core APIs produces code that is lightweight and readable. When necessary you can always use USD APIs to direct the stage.


# 5.Core API
[Core API tutorials](https://docs.omniverse.nvidia.com/isaacsim/latest/core_api_tutorials/index.html)